In [108]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [109]:
# Import indicator dataframe
df = pd.read_csv("../algotrader55/resources/aapl_15min_indc_df.csv")
df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,new_signal
0,2023-01-09 21:30:00+00:00,130.0817,130.15,130.0400,314,130.09,18713,130.096350,0.000167,130.19634,...,-2.688380,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0,1
1,2023-01-09 21:45:00+00:00,130.1000,130.14,130.1000,182,130.11,4711,130.121440,0.000141,130.11634,...,-2.717707,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0,1
2,2023-01-09 22:00:00+00:00,130.0600,130.12,130.0300,128,130.12,6111,130.076383,-0.000307,130.09034,...,-2.637492,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0,1
3,2023-01-09 22:15:00+00:00,130.0700,130.09,130.0315,210,130.06,95004,130.135637,0.000077,130.07434,...,-2.482767,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0,1
4,2023-01-09 22:30:00+00:00,130.0600,130.08,130.0300,148,130.04,4724,130.045182,-0.000077,130.07434,...,-2.311230,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0,1


In [110]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('new_signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("new_signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-09 21:45:00+00:00,130.0817,130.15,130.0400,314.0,130.09,18713.0,130.096350,0.000167,130.19634,130.50517,...,-104.945666,-2.688380,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0
2023-01-09 22:00:00+00:00,130.1000,130.14,130.1000,182.0,130.11,4711.0,130.121440,0.000141,130.11634,130.36567,...,-93.627848,-2.717707,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0
2023-01-09 22:15:00+00:00,130.0600,130.12,130.0300,128.0,130.12,6111.0,130.076383,-0.000307,130.09034,130.26267,...,-88.721576,-2.637492,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0
2023-01-09 22:30:00+00:00,130.0700,130.09,130.0315,210.0,130.06,95004.0,130.135637,0.000077,130.07434,130.20267,...,-82.039345,-2.482767,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0
2023-01-09 22:45:00+00:00,130.0600,130.08,130.0300,148.0,130.04,4724.0,130.045182,-0.000077,130.07434,130.17167,...,-76.985617,-2.311230,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0


Start date: 2023-01-09 21:45:00+00:00
End date: 2023-10-09 21:45:00+00:00


In [111]:
# Create a StandardScaler instance
scaler = StandardScaler()
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(11440, 97)

(5889, 97)

In [112]:
# NEURAL NETWORK
num_predictors = len(X.columns)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

nn_model = Sequential()


# Add dense layer(s)
nn_model.add(Dense(units=32, input_dim=num_predictors, activation='relu'))
nn_model.add(Dense(units=64, activation='relu'))
nn_model.add(Dense(units=128, activation='relu'))
nn_model.add(Dense(units=256, activation='relu'))
nn_model.add(Dense(units=128, activation='relu'))
nn_model.add(Dense(units=64, activation='relu'))
nn_model.add(Dense(units=32, activation='relu'))

# Drop-out layer(s)
nn_model.add(Dropout(.2,input_shape=(10,)))

# # Add dense layer, add Regularization
# nn_model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

# Add output layer
# Number of outputs equals number of classes
nn_model.add(Dense(num_classes, activation="sigmoid"))

In [113]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 32)                3136      
                                                                 
 dense_17 (Dense)            (None, 64)                2112      
                                                                 
 dense_18 (Dense)            (None, 128)               8320      
                                                                 
 dense_19 (Dense)            (None, 256)               33024     
                                                                 
 dense_20 (Dense)            (None, 128)               32896     
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dense_22 (Dense)            (None, 32)               

In [114]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100
92/92 [==============================] - 0s 2ms/step - loss: 0.6803 - accuracy: 0.5655 - val_loss: 0.6745 - val_accuracy: 0.5629
Epoch 2/100
92/92 [==============================] - 0s 2ms/step - loss: 0.6683 - accuracy: 0.5924 - val_loss: 0.6828 - val_accuracy: 0.5787
Epoch 3/100
92/92 [==============================] - 0s 2ms/step - loss: 0.6593 - accuracy: 0.6023 - val_loss: 0.7042 - val_accuracy: 0.5804
Epoch 4/100
92/92 [==============================] - 0s 2ms/step - loss: 0.6461 - accuracy: 0.6212 - val_loss: 0.7536 - val_accuracy: 0.5826
Epoch 5/100
92/92 [==============================] - 0s 3ms/step - loss: 0.6313 - accuracy: 0.6345 - val_loss: 0.8909 - val_accuracy: 0.5621
Epoch 6/100
92/92 [==============================] - 0s 2ms/step - loss: 0.6244 - accuracy: 0.6490 - val_loss: 0.8197 - val_accuracy: 0.5822
Epoch 7/100
92/92 [==============================] - 0s 2ms/step - loss: 0.6126 - accuracy: 0.6545 - val_loss: 0.8862 - val_accuracy: 0.5765
Epoch 8/100
9

In [115]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

185/185 - 0s - loss: 3.2250 - accuracy: 0.5407 - 103ms/epoch - 555us/step
Loss: 3.2250351905822754, Accuracy: 0.5406690239906311


In [116]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

358/358 [==============================] - 0s 430us/step


In [117]:
# Convert probabilities to class labels (0 or 1) using 0.5 as the threshold
nn_test_predictions_labels = (nn_test_predictions > 0.5).astype(int)
nn_train_predictions_labels = (nn_train_predictions > 0.5).astype(int)

# Training classification report
train_class_report = classification_report(y_train, nn_train_predictions_labels)
print(train_class_report)

# Testing classification report
test_class_report = classification_report(y_test, nn_test_predictions_labels)
print(test_class_report)

              precision    recall  f1-score   support

           0       0.92      0.87      0.89      5861
           1       0.87      0.92      0.89      5579

    accuracy                           0.89     11440
   macro avg       0.89      0.89      0.89     11440
weighted avg       0.89      0.89      0.89     11440

              precision    recall  f1-score   support

           0       0.57      0.53      0.55      3091
           1       0.52      0.55      0.53      2798

    accuracy                           0.54      5889
   macro avg       0.54      0.54      0.54      5889
weighted avg       0.54      0.54      0.54      5889



In [118]:
nn_test_predictions_labels

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [146]:
# Get the money associated with our signals... Start with $100k and buy/sell based on the signal generated by the algorithm. 
# 1) buy
# 2) wait
# 3) sell
# 4) calculate difference in price
nn_test_predictions_labels[0][0]

flattened_test_list = [item for sublist in nn_test_predictions_labels for item in sublist]

flattened_train_list = [item for sublist in nn_train_predictions_labels for item in sublist]


print(flattened_test_list[:10])
print(flattened_train_list[:10])

flat_list = flattened_test_list + flattened_train_list

flat_list[:10]

[1, 0, 0, 1, 1, 0, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


[1, 0, 0, 1, 1, 0, 0, 1, 1, 1]

In [162]:
# 0 to 1 is a buy signal, so label that 1 ->>>> 1 for buy
# 1 to 0 is sell                    ->>>> -1 for sell
# 0 to 0 and 1 to 1 is hold         ->>> 0 for hold

# signal_list starts with no position
signal_list = [0]

# Start counter for moving through list
y = 0

for x in flat_list[:len(flat_list)-1]:
    if flat_list[y] - flat_list[y+1] == 0:
        x = 0
    elif flat_list[y] - flat_list[y+1] == -1:
        x = 1
    else: 
        x = -1
    y+=1
    signal_list.append(x)


In [163]:
print(signal_list[:10])
print(signal_list[-30:])
print(len(signal_list))
print(flat_list[:10])
print(flat_list[-30:])
print(len(flat_list))


[0, -1, 0, 1, 0, -1, 0, 1, 0, 0]
[0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17329
[1, 0, 0, 1, 1, 0, 0, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
17329


Looks good.

In [173]:
# Now to backtest with money attatched
df = pd.read_csv("../algotrader55/resources/aapl_15min_indc_df.csv")

df
# start with no position
df.set_index('timestamp', inplace=True)
df

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,new_signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-09 21:30:00+00:00,130.0817,130.15,130.0400,314,130.09,18713,130.096350,0.000167,130.19634,130.50517,...,-2.688380,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0,1
2023-01-09 21:45:00+00:00,130.1000,130.14,130.1000,182,130.11,4711,130.121440,0.000141,130.11634,130.36567,...,-2.717707,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0,1
2023-01-09 22:00:00+00:00,130.0600,130.12,130.0300,128,130.12,6111,130.076383,-0.000307,130.09034,130.26267,...,-2.637492,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0,1
2023-01-09 22:15:00+00:00,130.0700,130.09,130.0315,210,130.06,95004,130.135637,0.000077,130.07434,130.20267,...,-2.482767,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0,1
2023-01-09 22:30:00+00:00,130.0600,130.08,130.0300,148,130.04,4724,130.045182,-0.000077,130.07434,130.17167,...,-2.311230,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-28 23:45:00+00:00,181.1200,181.20,181.1000,258,181.10,12027,181.143472,-0.000055,181.15498,181.13462,...,0.296206,2.131065,-0.677573,False,-2.712085e+07,0.000000,-7.472890,0.265597,100.0,1
2024-02-29 00:00:00+00:00,181.2000,181.20,181.0800,183,181.08,6942,181.152646,0.000442,181.15898,181.14062,...,0.245875,10.234980,-0.505418,False,-2.710697e+07,0.000000,-7.468373,0.266739,100.0,0
2024-02-29 00:15:00+00:00,181.3300,181.33,181.1500,163,181.20,4318,181.260887,0.000717,181.19098,181.15844,...,0.211222,21.909970,-0.025621,False,-2.710073e+07,0.000000,-7.466988,0.255867,100.0,0


In [175]:
df['new_signal'] = flat_list

df['buy sell signal'] = signal_list

In [177]:
df[['new_signal', 'buy sell signal']]


,new_signal,buy sell signal
timestamp,,
2023-01-09 21:30:00+00:00,1,0
2023-01-09 21:45:00+00:00,0,-1
2023-01-09 22:00:00+00:00,0,0
2023-01-09 22:15:00+00:00,1,1
2023-01-09 22:30:00+00:00,1,0
...,...,...
2024-02-28 23:45:00+00:00,1,0
2024-02-29 00:00:00+00:00,1,0
2024-02-29 00:15:00+00:00,1,0


In [178]:
# Now, for each element in 'buy sell signal' we need to get APPL's price at that time, then either buy or sell based on our signal.
# Let's just focus on longing the stock right now
# Multiply 

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,new_signal,buy sell signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-09 21:30:00+00:00,130.0817,130.15,130.0400,314,130.09,18713,130.096350,0.000167,130.19634,130.50517,...,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0,1,0
2023-01-09 21:45:00+00:00,130.1000,130.14,130.1000,182,130.11,4711,130.121440,0.000141,130.11634,130.36567,...,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0,0,-1
2023-01-09 22:00:00+00:00,130.0600,130.12,130.0300,128,130.12,6111,130.076383,-0.000307,130.09034,130.26267,...,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0,0,0
2023-01-09 22:15:00+00:00,130.0700,130.09,130.0315,210,130.06,95004,130.135637,0.000077,130.07434,130.20267,...,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0,1,1
2023-01-09 22:30:00+00:00,130.0600,130.08,130.0300,148,130.04,4724,130.045182,-0.000077,130.07434,130.17167,...,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-28 23:45:00+00:00,181.1200,181.20,181.1000,258,181.10,12027,181.143472,-0.000055,181.15498,181.13462,...,2.131065,-0.677573,False,-2.712085e+07,0.000000,-7.472890,0.265597,100.0,1,0
2024-02-29 00:00:00+00:00,181.2000,181.20,181.0800,183,181.08,6942,181.152646,0.000442,181.15898,181.14062,...,10.234980,-0.505418,False,-2.710697e+07,0.000000,-7.468373,0.266739,100.0,1,0
2024-02-29 00:15:00+00:00,181.3300,181.33,181.1500,163,181.20,4318,181.260887,0.000717,181.19098,181.15844,...,21.909970,-0.025621,False,-2.710073e+07,0.000000,-7.466988,0.255867,100.0,1,0


Connecting to ALPACA

In [123]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Load .env environment variables
load_dotenv()

True

In [124]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

In [125]:
# Set signal variable
signal = -1

# Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
elif signal == -1:
    orderSide = "sell"
else:
    orderSide = "hold"

In [126]:
# Set the ticket symbol and the number of shares to buy
ticker = "AAPL"
number_of_shares = 1

In [127]:
# Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["AAPL"])

# Get final closing price
limit_amount = prices["AAPL"]["close"][-1]

In [128]:
# Submit order
api.submit_order(
    symbol="AAPL", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="gtc", 
    type="market", 
    #limit_price=round(limit_amount, 3)
)

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': '25a15306-de1d-4fbd-b03f-f8b1ee0cddd8',
    'created_at': '2024-02-29T19:38:56.437268646Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'b0804561-c363-4e36-be43-155992304cb2',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2024-02-29T19:38:56.436797656Z',
    'subtag': None,
    'symbol': 'AAPL',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2024-02-29T19:38:56.437309696Z'})

OK, it's connected. Now we have to make order update when our algorithm tells it to.

Apparently, the free data we are getting from Alpaca is delayed by 15 minutes...
So how are we going to implement our live trading strategy on 15 minute delayed data?

In [129]:
# Load .env enviroment variables
load_dotenv()

True

In [130]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca_api_base_url = os.getenv("ALPACA_API_BASE_URL")

In [131]:
import asyncio

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, alpaca_api_base_url)
conn = tradeapi.stream.Stream(alpaca_api_key, alpaca_secret_key, base_url=alpaca_api_base_url, data_feed='iex')  # or use 'sip' for the paid data feed

async def on_quote(q):
    print('Quote:', q)

# This function schedules your coroutine to run on the existing event loop
async def main():
    conn.subscribe_quotes(on_quote, 'AAPL')
    await conn.run()

# Get the current event loop
loop = asyncio.get_event_loop()

# Schedule the main coroutine to run on the existing loop
# If using Python 3.7+, you can use asyncio.create_task() instead of ensure_future()
loop.create_task(main())

# If you're in an interactive environment like Jupyter and the loop is already running,
# you don't need to start the loop yourself. If you're in a script, you might need to use loop.run_forever()


<Task pending name='Task-12' coro=<main() running at /var/folders/tl/pmdc2_wx3hx1zkjc2fzswyyc0000gn/T/ipykernel_3104/3495666239.py:10>>

In [132]:
import asyncio

async def my_coroutine():
    # Your asynchronous code here
    print("Hello, asyncio!")

# Get the current event loop
loop = asyncio.get_event_loop()

# Schedule the coroutine to run on the existing loop
task = loop.create_task(my_coroutine())

# No need to start the loop manually in interactive environments


In [133]:
await main()  # Directly await the task if possible


Hello, asyncio!


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, alpaca_api_base_url)
# conn = tradeapi.stream.Stream(alpaca_api_key, alpaca_secret_key, base_url=alpaca_api_base_url, data_feed='iex')  # or use 'sip' for the paid data feed

# async def on_quote(q):
#     print('Quote:', q)

# # Subscribe to quotes for AAPL
# conn.subscribe_quotes(on_quote, 'AAPL')

# # Start the connection
# conn.run()


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# Example to subscribe to real-time updates
async def on_quote(q):
    print('Quote:', q)

conn = tradeapi.stream2.StreamConn()
conn.subscribe_quotes(on_quote, 'AAPL')


AttributeError: module 'alpaca_trade_api' has no attribute 'stream2'